In [1]:
from coffea import processor, util

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
import numpy as np
from coffea import processor,hist
from uproot_methods import TLorentzVectorArray
from boostedhiggs.common import getParticles
import warnings
import awkward
import os

In [3]:
class HwwProcessor_Trig(processor.ProcessorABC):
    def __init__(self, year):
        self._year = year
        self._triggers = {
            '2017_had': [
                "AK8PFJet330_PFAK8BTagCSV_p17",
                "PFHT1050",
                "AK8PFJet400_TrimMass30",
                "AK8PFJet420_TrimMass30",
                "AK8PFHT800_TrimMass50",
                "PFJet500",
                "AK8PFJet500",
                ],
            '2017_muon': [
                "Mu50","Mu55"
                ],
            '2017_vvlmuon':[
                "Mu15_IsoVVVL_PFHT450_PFMET50","Mu15_IsoVVVL_PFHT600",
                ],
            '2017_electron': [
                "Ele27_WPTight_Gsf","Ele40_WPTight_Gsf","Ele20_WPLoose_Gsf",
                "Ele115_CaloIdVT_GsfTrkIdT"],
            '2017_vvlelectron': [
                "Ele15_IsoVVVL_PFHT450_PFMET50","Ele15_IsoVVVL_PFHT600",
                ],
            '2017_met':[
                "PFMETNoMu110_PFMHTNoMu110_IDTight",
                "PFMETNoMu120_PFMHTNoMu120_IDTight_PFHT60",
                "PFMETNoMu120_PFMHTNoMu120_IDTight",
                ],
            }

        self._triggers['2017_muonall'] = self._triggers['2017_muon']+self._triggers['2017_had']+self._triggers['2017_met']+self._triggers['2017_vvlmuon']
        self._triggers['2017_electronall'] = self._triggers['2017_electron']+self._triggers['2017_had']+self._triggers['2017_met']+self._triggers['2017_vvlelectron']

        self._triggers['2017_muonnomet'] = self._triggers['2017_muon']+self._triggers['2017_had']+self._triggers['2017_vvlmuon']
        self._triggers['2017_electronnomet'] = self._triggers['2017_electron']+self._triggers['2017_had']+self._triggers['2017_vvlelectron']

    
        dataset_axis = hist.Cat("dataset", "Primary dataset")
        jetpt_axis = hist.Bin("jet_pt", r"Jet $p_T$", 30, 250, 800)
        genHpt_axis = hist.Bin("genH_pt", r"gen Higgs $p_T$",30, 250, 800)
        genElept_axis = hist.Bin("genEle_pt", r"gen Electron $p_T$",30, 10, 70)
        genMupt_axis = hist.Bin("genMu_pt", r"gen Muon $p_T$",30, 10, 70)

        hists = processor.dict_accumulator()
        hist.Hist.DEFAULT_DTYPE = 'f'
        hists['sumw'] = processor.defaultdict_accumulator(int)
        hists['cutflow_ele'] = processor.defaultdict_accumulator(float)
        hists['cutflow_muon'] = processor.defaultdict_accumulator(float)
        for sel in ['presel','noniso']:
            hists['Ele'+sel] = hist.Hist("Events / GeV",
                                          dataset_axis,
                                          jetpt_axis,
                                          genHpt_axis,
                                          genElept_axis,
                                          )
            hists['Mu'+sel] = hist.Hist("Events / GeV",
                                         dataset_axis,
                                         jetpt_axis,
                                         genHpt_axis,
                                         genMupt_axis,
                                         )
            for key in self._triggers.keys():
                key = sel+'_'+key.replace('_','')
                hists['Ele'+key] = hist.Hist("Events / GeV",
                                             dataset_axis,
                                             jetpt_axis,
                                             genHpt_axis,
                                             genElept_axis,
                                             )
                hists['Mu'+key] = hist.Hist("Events / GeV",
                                            dataset_axis,
                                            jetpt_axis,
                                            genHpt_axis,
                                            genMupt_axis,
                                            )
        self._accumulator = hists

    @property
    def accumulator(self):
        return self._accumulator

    def process(self, events):
        dataset = events.metadata['dataset']
        output = self.accumulator.identity()
        selection = processor.PackedSelection()
        output = self.accumulator.identity()

        # trigger                                                                                                                                                                    
        for key,item in self._triggers.items():
            trigger = np.zeros(events.size, dtype='bool')
            for t in item:
                trigger = trigger | events.HLT[t]
            selection.add(key, trigger)

        # Muons                                                                                                                                                                      
        goodMuon = (
            (events.Muon.pt > 20.)
            & (np.abs(events.Muon.eta) < 2.4)
            )
        nmuons = goodMuon.sum()

        # one Muon                                                                                                                                                                   
        selection.add('onemuon', (nmuons >= 1))

        # Electrons                                                                                                                                                                  
        goodElectron = (
            (events.Electron.pt > 20.)
            & (np.abs(events.Electron.eta) < 2.5)
            )
        nelectrons = goodElectron.sum()

        # one Electron                                                                                                                                                               
        selection.add('oneelectron', (nelectrons >= 1))
        
        # select FatJet with muon closest                                                                                                                                            
        goodFatJet = (
            (events.FatJet.pt > 300.)
            & (np.abs(events.FatJet.eta) < 2.4)
            & (events.FatJet.msoftdrop > 10.)
            & (events.FatJet.jetId & 2)
            )
        leadingmuon = events.Muon[goodMuon][:, 0:1]
        leadingele = events.Electron[goodElectron][:, 0:1]
        leadingjet = events.FatJet[goodFatJet][:,0:1]

        selection.add('jetkin', (leadingjet.pt > 300).any())
        selection.add('nonisomuon', (leadingmuon.pt > 50).any())
        selection.add('nonisoelectron', (leadingmuon.pt > 115).any())

        # gen matching                                                                                                                                                               
        genH,genH_idx = getParticles(events,25,['fromHardProcess', 'isLastCopy'])
        genW,genW_idx = getParticles(events,24,['fromHardProcess', 'isLastCopy'])
        genE,genE_idx = getParticles(events,11,['fromHardProcess','isFirstCopy'],1)
        genM,genM_idx = getParticles(events,13,['fromHardProcess','isFirstCopy'],1)
        genT,genT_idx = getParticles(events,15,['fromHardProcess','isFirstCopy'],1)
        genQ,genQ_idx = getParticles(events,[0,5],['fromHardProcess','isFirstCopy'])

        ishWW_qqelev = (genH.counts==1) & (genW.counts==2) & (genE.counts==1) & (genM.counts==0) & (genT.counts==0)
        ishWW_qqmuv = (genH.counts==1) & (genW.counts==2) & (genM.counts==1) & (genE.counts==0) & (genT.counts==0)
        ishWW_qqtauv = (genH.counts==1) & (genW.counts==2) & (genT.counts==1) & (genM.counts==0) & (genE.counts==0)
        ishWW_qqqq = (genH.counts==1) & (genW.counts==2) & (genQ.counts==4)  & (genM.counts==0) & (genE.counts==0)
        ishWW_muvelev = (genH.counts==1) & (genW.counts==2) & (genE.counts==1) & (genM.counts==1)
        ishWW_elevelev = (genH.counts==1) & (genW.counts==2) & (genE.counts==2) & (genM.counts==0)
        ishWW_tauvtauv = (genH.counts==1) & (genW.counts==2) & (genT.counts==2) & (genM.counts==0) & (genE.counts==0)
        ishWW_muvmuv = (genH.counts==1) & (genW.counts==2) & (genE.counts==0) & (genM.counts==2)

        selection.add('genEle',ishWW_qqelev)
        selection.add('genMu',ishWW_qqmuv)
        
        regions = {}
        for key in self._triggers.keys():
            regions[key.replace('_','')] = {key}
        regions['Elepresel'] = {'jetkin', 'oneelectron','genEle'}
        regions['Mupresel'] = {'jetkin', 'onemuon','genMu'}
        regions['Elenoniso'] = {'jetkin', 'oneelectron','genEle','nonisoelectron'}
        regions['Munoniso'] = {'jetkin', 'onemuon','genMu','nonisomuon'}

        #cutflow                                                                                                                                                                     
        cutflow_ele = ['genEle', 'jetkin', 'oneelectron', 'nonisoelectron']
        cutflow_mu = ['genMu', 'jetkin', 'onemuon', 'nonisomuon']
        allcuts = set()
        output['cutflow_ele']['none'] += len(events)
        output['cutflow_muon']['none'] += len(events)
        for cut in cutflow_ele:
            allcuts.add(cut)
            output['cutflow_ele'][cut] += selection.all(*allcuts).sum()
        allcuts = set()
        for cut in cutflow_mu:
            allcuts.add(cut)
            output['cutflow_muon'][cut] += selection.all(*allcuts).sum()
        
        weights = processor.Weights(len(events))
        weights.add('genweight', events.genWeight)
        
        for histname, h in output.items():
            if not isinstance(h, hist.Hist):
                continue
            region = []
            for r in regions.keys():
                if len(histname.split('_'))> 2:
                    histname2 = histname
                else:
                    histname2 = histname.split('_')
                if r in histname2:
                    region.extend(regions[r])
            cut = selection.all(*region)
            weight = np.ones(events.size, dtype='bool')
            #weight = weights.weight()
            #weight = cut
            
            def normalize(val):
                return val.pad(1, clip=True).fillna(0).flatten()

            if 'Mu' in histname:
                h.fill(
                    jet_pt = normalize(leadingjet.pt),
                    genH_pt = normalize(genH.pt),
                    genMu_pt = normalize(genM.pt),
                    dataset=dataset,
                    weight=weight*cut)
            else:
                h.fill(
                    jet_pt = normalize(leadingjet.pt),
                    genH_pt = normalize(genH.pt),
                    genEle_pt = normalize(genE.pt),
                    dataset=dataset,
                    weight=weight*cut)

        return output
    
    def postprocess(self, accumulator):
        return accumulator

In [4]:
files = {
    #'GluGluHToWWToLNuQQ_M125_TuneCP5_PSweight_13TeV-powheg2-jhugen727-pythia8': [
    #    'data/GluGluHToWWToLNuQQ_M125_TuneCP5_PSweight_13TeV-powheg2-jhugen727-pythia8-hadd/nano_mc_2017_0.root',
    #    'data/GluGluHToWWToLNuQQ_M125_TuneCP5_PSweight_13TeV-powheg2-jhugen727-pythia8-hadd/nano_mc_2017_1.root',
    #    'data/GluGluHToWWToLNuQQ_M125_TuneCP5_PSweight_13TeV-powheg2-jhugen727-pythia8-hadd/nano_mc_2017_2.root',
    #    'data/GluGluHToWWToLNuQQ_M125_TuneCP5_PSweight_13TeV-powheg2-jhugen727-pythia8-hadd/nano_mc_2017_3.root',
    #    'data/GluGluHToWWToLNuQQ_M125_TuneCP5_PSweight_13TeV-powheg2-jhugen727-pythia8-hadd/nano_mc_2017_4.root',
    #    'data/GluGluHToWWToLNuQQ_M125_TuneCP5_PSweight_13TeV-powheg2-jhugen727-pythia8-hadd/nano_mc_2017_5.root',
    #    'data/GluGluHToWWToLNuQQ_M125_TuneCP5_PSweight_13TeV-powheg2-jhugen727-pythia8-hadd/nano_mc_2017_6.root',
    #    'data/GluGluHToWWToLNuQQ_M125_TuneCP5_PSweight_13TeV-powheg2-jhugen727-pythia8-hadd/nano_mc_2017_7.root',
    #    'data/GluGluHToWWToLNuQQ_M125_TuneCP5_PSweight_13TeV-powheg2-jhugen727-pythia8-hadd/nano_mc_2017_8.root',
    #    'data/GluGluHToWWToLNuQQ_M125_TuneCP5_PSweight_13TeV-powheg2-jhugen727-pythia8-hadd/nano_mc_2017_9.root'
    #],
    'gghww': ['data/hww_2017mc_UL-hadd/nano_RunIISummer19UL17MiniAOD_Feb10.root']
}

p = HwwProcessor_Trig(year='2017')

exe_config = {
    'flatten': True,
    'workers': 4,
    'savemetrics': True,
    'compression': 0,
    'nano': True,
}

from pyinstrument import Profiler
profiler = Profiler()
profiler.start()
output, metrics = processor.run_uproot_job(files, 'Events', p, processor.iterative_executor, exe_config)
profiler.stop()

evtrate = metrics['entries'].value / metrics['processtime'].value
print("Events / s / thread: %.0f" % evtrate)
print("Columns accessed:", set(metrics['columns']))


Events / s / thread: 3648
Columns accessed: {'GenPart_status', 'HLT_Ele20_WPLoose_Gsf', 'nElectron', 'nMuon', 'GenPart_pt', 'HLT_AK8PFJet330_PFAK8BTagCSV_p17', 'HLT_Ele40_WPTight_Gsf', 'Muon_pt', 'HLT_PFMETNoMu120_PFMHTNoMu120_IDTight_PFHT60', 'HLT_Mu50', 'genWeight', 'FatJet_msoftdrop', 'HLT_Ele27_WPTight_Gsf', 'HLT_Ele115_CaloIdVT_GsfTrkIdT', 'HLT_AK8PFJet420_TrimMass30', 'HLT_Ele15_IsoVVVL_PFHT600', 'HLT_Mu55', 'FatJet_pt', 'GenPart_statusFlags', 'HLT_AK8PFJet500', 'GenPart_pdgId', 'FatJet_jetId', 'HLT_AK8PFJet400_TrimMass30', 'nGenPart', 'HLT_Mu15_IsoVVVL_PFHT450_PFMET50', 'FatJet_eta', 'HLT_Mu15_IsoVVVL_PFHT600', 'HLT_PFHT1050', 'HLT_PFMETNoMu110_PFMHTNoMu110_IDTight', 'HLT_PFMETNoMu120_PFMHTNoMu120_IDTight', 'HLT_PFJet500', 'Muon_eta', 'nFatJet', 'Electron_eta', 'HLT_Ele15_IsoVVVL_PFHT450_PFMET50', 'HLT_AK8PFHT800_TrimMass50', 'Electron_pt'}


In [5]:
from boostedhiggs import processmap
hists_unmapped = output
hists_mapped = {}
for key, val in hists_unmapped.items():
    if isinstance(val, hist.Hist):
        hists_mapped[key] = processmap.apply(val)

/Users/cristina/Library/Python/3.6/lib/python/site-packages/coffea/hist/hist_tools.py:346: RuntimeWarning: Not all requested indices present in <Cat (name=dataset) instance at 0x11ca5a9b0>
  warnings.warn("Not all requested indices present in %r" % self, RuntimeWarning)


In [6]:
def drawExcept(h,axis_label,fig_lab='',overlay_label=None,title_lab=None,integrate_sel=None):
    h1 = h
    axisDict = {axis_label}
    if overlay_label is not None:
        axisDict = {axis_label,overlay_label}
    if integrate_sel is not None:
        h1 = h.integrate(integrate_sel[0],integrate_sel[1])
    all_bkg=0
    for key1,val1 in h1.values().items():
        all_bkg+=val1.sum()
    print(all_bkg)
    fig,axis = plt.subplots(1,1, figsize=(8,8))
    hist.plot1d(h1.sum(*[ax for ax in h1.axes() if ax.name not in axisDict]),overlay=overlay_label,ax=axis)
    if title_lab is not None:
        leg = ax.get_legend()
        leg.set_title(title_lab)
    fig.savefig("plots/%s%s.pdf"%(fig_lab,axis_label))
    #plt.show()
    plt.clf()

In [7]:
def drawEfficiency(hnum,hdenom,axis_label,fig_lab='',title='',odir=''):
    data_err_opts = {
    'linestyle':'none',
    'marker': '.',
    'markersize': 10.,
    'color':'k',
    'elinewidth': 1,
    'emarker': '_'
    }
    axisDict = {axis_label}
    fig,axis = plt.subplots(1,1, figsize=(8,8))
    hist.plotratio(num=hnum.sum(*[ax for ax in hnum.axes() if ax.name not in axisDict]),
                   denom=hdenom.sum(*[ax for ax in hdenom.axes() if ax.name not in axisDict]),
                   error_opts=data_err_opts,
                   guide_opts={},
                   ax=axis,
                   #unc='poisson-ratio')
                   unc='clopper-pearson')                                                                                                                                
    axis.set_ylim(0.2,1.2)
    axis.set_title(title)
    fig.savefig("%s/triggereff_%s.pdf"%(odir,fig_lab))
    #plt.show()
    plt.clf()

In [8]:
odir = '../binder/plots/trigger_hww_private_Mar26_leppt20'
os.system('mkdir -p %s'%odir)
denoms = ['presel','noniso']
#drawExcept(hists_mapped['Mupresel'],'genMu_pt')
#drawExcept(hists_mapped['Mupresel_2017had'],'genMu_pt')
#drawEfficiency(hists_mapped['Mupresel_2017had'],
#               hists_mapped['Mupresel'],
#               'genH_pt')
for denom in denoms:
    elekey = 'Ele'+denom
    mukey = 'Mu'+denom
    trigger_plots = {
        mukey+'_hadtrig': {'vars':['genH_pt','genMu_pt','jet_pt'],
                       'denom':mukey,
                       'num':mukey+'_2017had',
                       'title':'PFHT | AK8*_TrimMass',},
        mukey+'_mutrig': {'vars':['genH_pt','genMu_pt','jet_pt'],
                       'denom':mukey,
                       'num':mukey+'_2017muon',
                       'title':'Mu50 | Mu55',},
        mukey+'_vvlmutrig': {'vars':['genH_pt','genMu_pt','jet_pt'],
                       'denom':mukey,
                       'num':mukey+'_2017vvlmuon',
                       'title':'Mu15VVL_HT/MET',},
        mukey+'_mettrig': {'vars':['genH_pt','genMu_pt','jet_pt'],
                       'denom':mukey,
                       'num':mukey+'_2017met',
                       'title':'PFMETNoMu110/120',},
        mukey+'_alltrig': {'vars':['genH_pt','genMu_pt','jet_pt'],
                       'denom':mukey,
                       'num':mukey+'_2017muonall',
                       'title':'Mu All',},
        mukey+'_nomettrig': {'vars':['genH_pt','genMu_pt','jet_pt'],
                       'denom':mukey,
                       'num':mukey+'_2017muonnomet',
                       'title':'Mu All - MET',},
        elekey+'_hadtrig': {'vars':['genH_pt','genEle_pt','jet_pt'],
                       'denom':elekey,
                       'num':elekey+'_2017had',
                       'title':'PFHT | AK8*_TrimMass',},
        elekey+'_eletrig': {'vars':['genH_pt','genEle_pt','jet_pt'],
                       'denom':elekey,
                       'num':elekey+'_2017electron',
                       'title':'Ele115',},
        elekey+'_vvlmutrig': {'vars':['genH_pt','genEle_pt','jet_pt'],
                       'denom':elekey,
                       'num':elekey+'_2017vvlelectron',
                       'title':'EleVVL_HT/MET',},
        elekey+'_mettrig': {'vars':['genH_pt','genEle_pt','jet_pt'],
                       'denom':elekey,
                       'num':elekey+'_2017met',
                       'title':'PFMETNoMu110/120',},
        elekey+'_alltrig': {'vars':['genH_pt','genEle_pt','jet_pt'],
                       'denom':elekey,
                       'num':elekey+'_2017electronall',
                       'title':'Ele All',},
        elekey+'_nomettrig': {'vars':['genH_pt','genEle_pt','jet_pt'],
                       'denom':elekey,
                       'num':elekey+'_2017electronnomet',
                       'title':'Ele All - MET',},
    }
    for key,item in trigger_plots.items():
        h_num = hists_unmapped[item['num']]
        h_denom = hists_unmapped[item['denom']]
        for var in item['vars']:
                drawEfficiency(
                    h_num,h_denom,var,
                    fig_lab=key+'_'+var,title=item['title'],
                    odir=odir
                )

/Users/cristina/Library/Python/3.6/lib/python/site-packages/mplhep/__init__.py:49: MatplotlibDeprecationWarning: 
The createFontList function was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use FontManager.addfont instead.
  font_list = fm.createFontList(font_files)
/Users/cristina/Library/Python/3.6/lib/python/site-packages/coffea/hist/plot.py:359: RuntimeWarning: invalid value encountered in true_divide
  rsumw = sumw_num / sumw_denom
/Users/cristina/Library/Python/3.6/lib/python/site-packages/coffea/hist/plot.py:359: RuntimeWarning: invalid value encountered in true_divide
  rsumw = sumw_num / sumw_denom
/Users/cristina/Library/Python/3.6/lib/python/site-packages/coffea/hist/plot.py:359: RuntimeWarning: invalid value encountered in true_divide
  rsumw = sumw_num / sumw_denom
/Users/cristina/Library/Python/3.6/lib/python/site-packages/coffea/hist/plot.py:359: RuntimeWarning: invalid value encountered in true_divide
  rsumw = sumw_num / sumw_denom
/Users

/Users/cristina/Library/Python/3.6/lib/python/site-packages/ipykernel_launcher.py:11: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  # This is added back by InteractiveShellApp.init_path()
/Users/cristina/Library/Python/3.6/lib/python/site-packages/coffea/hist/plot.py:359: RuntimeWarning: invalid value encountered in true_divide
  rsumw = sumw_num / sumw_denom
/Users/cristina/Library/Python/3.6/lib/python/site-packages/ipykernel_launcher.py:11: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  # This is added back by InteractiveShellApp.init_path()


/Users/cristina/Library/Python/3.6/lib/python/site-packages/ipykernel_launcher.py:11: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  # This is added back by InteractiveShellApp.init_path()
/Users/cristina/Library/Python/3.6/lib/python/site-packages/coffea/hist/plot.py:359: RuntimeWarning: invalid value encountered in true_divide
  rsumw = sumw_num / sumw_denom
/Users/cristina/Library/Python/3.6/lib/python/site-packages/ipykernel_launcher.py:11: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  # This is added back by InteractiveShellApp.init_path()


/Users/cristina/Library/Python/3.6/lib/python/site-packages/ipykernel_launcher.py:11: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  # This is added back by InteractiveShellApp.init_path()
/Users/cristina/Library/Python/3.6/lib/python/site-packages/coffea/hist/plot.py:359: RuntimeWarning: invalid value encountered in true_divide
  rsumw = sumw_num / sumw_denom
/Users/cristina/Library/Python/3.6/lib/python/site-packages/ipykernel_launcher.py:11: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  # This is added back by InteractiveShellApp.init_path()


<Figure size 576x576 with 0 Axes>

...

<Figure size 576x576 with 0 Axes>

<Figure size 576x576 with 0 Axes>

<Figure size 576x576 with 0 Axes>

<Figure size 576x576 with 0 Axes>

<Figure size 576x576 with 0 Axes>